# RDD : Resilient Distributed Datasets

In [1]:
#we use rdds in fewer cases , and should not depend on these as much as we can , 
#becasue these provides a lot of work , and lacks a lot of optimizations .that are available in structured 
#we use rdds when we want to control over the pysical distributions of data acoree the cluster

## create RDDs

In [2]:
# from dataframes and datasets ,by converting them to rdd 
spark.range(10).rdd

MapPartitionsRDD[5] at javaToPython at <unknown>:0

In [3]:
# create dataframe from range functions
spark.range(3).toDF("id").show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
+---+



In [4]:
#operate over these 
maprdd = spark.range(10).toDF("id").rdd.map(lambda row:row[0])
maprdd.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [5]:
# you can create the DF from rdd with the same methodology with toDF() function
spark.range(10).rdd.toDF()
# create rdd of type Row 

DataFrame[id: bigint]

In [6]:
# also we can create RDD from Collection , we need tp parallelize method in sparkcontext
#parallelize : turns single node collection  into parrallel collections
mycollection = "spark the defenitive guide :big data processing made simple".split(" ")
words = spark.sparkContext.parallelize(mycollection,2)
words.collect()

['spark',
 'the',
 'defenitive',
 'guide',
 ':big',
 'data',
 'processing',
 'made',
 'simple']

In [7]:
words2 = sc.parallelize(mycollection,2)
#bydefault spark session define sparkcontext  as sc , so these print the same results

In [8]:
# we can name the rdd , to show up the spark UI
words.setName("Mywords")
words.name()

'Mywords'

In [9]:
#myfile  = sc.textFile("filename") , this will create rdd 
#each record in rdd represent line in textfile 
#mycollection  = sc.wholeTextFiles("files") here name of the file is the first object and the value of the text file is the second object

## manipulate RDDs

In [10]:
# distinct 
words.distinct().count()

9

In [11]:
# filter : return boolen type to be used as filter function
def startswiths (ind):
    return ind.startswith("s")

words.filter(lambda word : startswiths(word)).collect()

['spark', 'simple']

In [12]:
#map
#spaecify function that retuen value and given correct input
wordss = words.map(lambda word : (word ,word[0] , word.startswith("s")) )
# this returns the (word ,word[0] , word.startswith("s") . word itself , 1st char , true or false

In [13]:
wordss.collect()

[('spark', 's', True),
 ('the', 't', False),
 ('defenitive', 'd', False),
 ('guide', 'g', False),
 (':big', ':', False),
 ('data', 'd', False),
 ('processing', 'p', False),
 ('made', 'm', False),
 ('simple', 's', True)]

In [14]:
#then we will filter wordss rdd to return only true
wordss.filter(lambda word : word[2]).collect()

[('spark', 's', True), ('simple', 's', True)]

In [15]:
#flatmap , each row should return multiple rows
# it needs the output of the map function be an iterable that can be expanded
words.flatMap(lambda word : list(word)).take(5)

['s', 'p', 'a', 'r', 'k']

In [16]:
#sortby *-1 to sort desc
words.sortBy(lambda word : len(word)*-1).take(2)

['defenitive', 'processing']

## Actions

In [17]:
# reduce an rdd any kind of value to one value 
sc.parallelize(range(1,21)).reduce(lambda x,y :x+y)

210

In [18]:
#longest word in our set of words 
def wordlengthreduce(leftword , rightword):
    if len(leftword) > len(rightword):
        return leftword
    else:
        return rightword

In [19]:
words.reduce(wordlengthreduce)

'processing'

In [20]:
words.count()

9

In [21]:
#countByValue(): counts the number of values in a given rdd , itload the entire dataset into memory.
# you should use it when the dataset is small 
words.countByValue()

defaultdict(int,
            {'spark': 1,
             'the': 1,
             'defenitive': 1,
             'guide': 1,
             ':big': 1,
             'data': 1,
             'processing': 1,
             'made': 1,
             'simple': 1})

In [22]:
#first : return the first itemin dataset
words.first()

'spark'

In [29]:
sc.parallelize(range (1,20)).max()

19

In [34]:
# take() and it's derivatives , this first scanning one partition and check the result to satisfy
# take , takeOrdered , takeSample , top 
print (words.take(5))
print (words.takeOrdered(5))
print (words.top(5))

['spark', 'the', 'defenitive', 'guide', ':big']
[':big', 'data', 'defenitive', 'guide', 'made']
['the', 'spark', 'simple', 'processing', 'made']


## Saving Files

In [35]:
# saving files , means that writing to plain text ,
#you must iterate over the partition in order to save the content to external database
# if we have more than one partition , will endup with more than one partition

In [41]:
words.saveAsTextFile("Finalwords2")

In [39]:
#sequencefiles : flat file consist of binary key-value pairs , it's used in map reduce jobs
# can create sequence files using saveAsObjectFile
#words.saveAsObjectFile("sequence")

## caching

In [42]:
#cache and persist , only handle data in memory, we can name it if we use setName function
words.cache()

Mywords ParallelCollectionRDD[28] at readRDDFromFile at PythonRDD.scala:247

In [43]:
#we can spaecify storage level (memory only , disk only , separately , offheap)
words.getStorageLevel()

StorageLevel(False, True, False, False, 1)

## checkpointing

In [44]:
#not available in dataframes , similar to caching except it's not stored in memory, only disk.
#helpful when performing iterative cmputation  , similar to caching
sc.setCheckpointDir("checkpointing")
words.checkpoint()
# when we reference this rdd , it will derive from checkpoint instead of source data